In [1]:
import re 
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, explode, lower, col, lit
from pyspark.sql.types import StringType, ArrayType

spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/30 00:44:34 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
22/04/30 00:44:34 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
22/04/30 00:44:34 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
22/04/30 00:44:34 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator


In [2]:
df = spark.read.csv('gs://hw2-jl-bucket/p1t20428.csv', sep = '\t')

In [3]:
def add_num_neighbors(df):
    num_neighbor = df.groupBy('_c0').count()
    df = df.join(num_neighbor, df['_c0'] == num_neighbor['_c0'], how='left').select(df['*'], num_neighbor['count'])
    return(df)

In [6]:
def PageRank(df):
    
    df = df.withColumn('contribute', df['rank']/df['count'])
#     df.show()
    
    total_contribution = df.groupBy('_c1').sum('contribute')
    total_contribution = total_contribution.select(col('_c1').alias('c1'), col('sum(contribute)').alias('contribute_sum'))
    df = df.join(total_contribution, df['_c0'] == total_contribution['c1'], how='left').select(df['*'], total_contribution['contribute_sum'])
#     df.show()
    
    df = df.withColumn( 'new_rank', df['contribute_sum'] * (17/20) + 3/20 )
#     df.show()

    df = df.select('_c0', '_c1', 'count', col('new_rank').alias('rank'))
#     df.show()
    
    return(df)

In [7]:
df = add_num_neighbors(df).withColumn('rank', lit(1))
df.show()

for it in range(10):
    df = PageRank(df)

df.show()

+----+--------------------+-----+----+
| _c0|                 _c1|count|rank|
+----+--------------------+-----+----+
|1090|                1153|   30|   1|
|1090|                1155|   30|   1|
|1090|             algeria|   30|   1|
|1090|   almoravid dynasty|   30|   1|
|1090|              bejaia|   30|   1|
|1090|   belt (mechanical)|   30|   1|
|1090|bernard of clairvaux|   30|   1|
|1090|       category:1090|   30|   1|
|1090|common year start...|   30|   1|
|1090|egbert ii, margra...|   30|   1|
|1090|  eliezer ben nathan|   30|   1|
|1090|             granada|   30|   1|
|1090|            hammadid|   30|   1|
|1090|jaromir (bishop o...|   30|   1|
|1090|     julian calendar|   30|   1|
|1090|              july 3|   30|   1|
|1090|             june 26|   30|   1|
|1090|               mainz|   30|   1|
|1090|           october 6|   30|   1|
|1090|            provence|   30|   1|
+----+--------------------+-----+----+
only showing top 20 rows



+---------+--------------------+-----+------------------+
|      _c0|                 _c1|count|              rank|
+---------+--------------------+-----+------------------+
|1 e+12 m²|           1 e+11 m²|  161|0.1588296263568943|
|1 e+12 m²|           1 e+13 m²|  161|0.1588296263568943|
|1 e+12 m²|               1e5 m|  161|0.1588296263568943|
|1 e+12 m²|               1e5 m|  161|0.1588296263568943|
|1 e+12 m²|               1e5 m|  161|0.1588296263568943|
|1 e+12 m²|               1e6 m|  161|0.1588296263568943|
|1 e+12 m²|              alaska|  161|0.1588296263568943|
|1 e+12 m²|             algeria|  161|0.1588296263568943|
|1 e+12 m²|        amazon river|  161|0.1588296263568943|
|1 e+12 m²|amazonas (brazili...|  161|0.1588296263568943|
|1 e+12 m²|                area|  161|0.1588296263568943|
|1 e+12 m²|           argentina|  161|0.1588296263568943|
|1 e+12 m²|        ariel (moon)|  161|0.1588296263568943|
|1 e+12 m²|               asean|  161|0.1588296263568943|
|1 e+12 m²|   

In [8]:
df = df.select(col('_c0').alias('article'), 'rank').distinct().dropna().sort(["article", "rank"],ascending=True).limit(5)
df.show()

+--------------------+-------------------+
|             article|               rank|
+--------------------+-------------------+
|         'ndrangheta|0.17283357863173981|
|    's-hertogenbosch| 0.1834913391892671|
|...baby one more ...|0.15137860346513293|
|          0 (number)|0.38543954827264343|
|                  0s| 0.1620937576107053|
+--------------------+-------------------+



In [9]:
gcs_filepath = 'gs://hw2-jl-bucket/p1t3_small_final.csv'
df.coalesce(1).write.option("delimiter", "\t").csv(gcs_filepath)